# LangChain with Open Source LLM and Open Source Embeddings & LangSmith

In the following notebook we will dive into the world of Open Source models hosted on Hugging Face's [inference endpoints](https://ui.endpoints.huggingface.co/).

The notebook will be broken into the following parts:

- 🤝 Breakout Room #1:
  1. Set-up Hugging Face Infrence Endpoints
  2. Install required libraries
  3. Set Environment Variables
  4. Testing our Hugging Face Inference Endpoint
  5. Creating LangChain components powered by the endpoints
  6. Retrieving data from Arxiv
  7. Creating a simple RAG pipeline with [LangChain v0.1.0](https://blog.langchain.dev/langchain-v0-1-0/)
  

- 🤝 Breakout Room #2:
  1. Set-up LangSmith
  2. Creating a LangSmith dataset
  3. Creating a custom evaluator
  4. Initializing our evaluator config
  5. Evaluating our RAG pipeline

# 🤝 Breakout Room #1

## Task 1: Set-up Hugging Face Infrence Endpoints

Please follow the instructions provided [here](https://github.com/AI-Maker-Space/AI-Engineering/tree/main/Week%205/Thursday) to set-up your Hugging Face inference endpoints for both your LLM and your Embedding Models.

## Task 2: Install required libraries

Now we've got to get our required libraries!

We'll start with our `langchain` and `huggingface` dependencies.



In [ ]:
!pip install langchain langchain-core langchain-community langchain_openai huggingface-hub requests -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

Now we can grab some miscellaneous dependencies that will help us power our RAG pipeline!

In [ ]:
!pip install arxiv pymupdf faiss-cpu -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 16.4 MB/s eta 0:00:00


## Task 3: Set Environment Variables

We'll need to set our `HF_TOKEN` so that we can send requests to our protected API endpoint.

We'll also set-up our OpenAI API key, which we'll leverage later.



In [ ]:
import os
import getpass

os.environ["HF_TOKEN"] = getpass.getpass("HuggingFace Write Token: ")

HuggingFace Write Token: ··········


In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


## Task 4: Testing our Hugging Face Inference Endpoint

Let's submit a sample request to the Hugging Face Inference endpoint!

In [ ]:
model_api_gateway = "https://owmg8v0a7eyh7qii.us-east-1.aws.endpoints.huggingface.cloud" # << YOUR ENDPOINT URL HERE

> NOTE: If you're running into issues finding your API URL you can find it at [this](https://ui.endpoints.huggingface.co/) link.

Here's an example:

![image](https://i.imgur.com/XyZhOv8.png)

In [ ]:
import requests

max_new_tokens = 256
top_p = 0.9
temperature = 0.1

prompt = "Hello! How are you?"

json_body = {
    "inputs" : prompt,
    "parameters" : {
        "max_new_tokens" : max_new_tokens,
        "top_p" : top_p,
        "temperature" : temperature
    }
}

headers = {
  "Authorization": f"Bearer {os.environ['HF_TOKEN']}",
  "Content-Type": "application/json"
}

response = requests.post(model_api_gateway, json=json_body, headers=headers)
print(response.json())

[{'generated_text': "Hello! How are you? I'm doing well, thanks for asking! *smiling*\n\n\n"}]


## Task 5: Creating LangChain components powered by the endpoints

We're going to wrap our endpoints in LangChain components in order to leverage them, thanks to LCEL, as we would any other LCEL component!

### HuggingFaceEndpoint for LLM

We can use the `HuggingFaceEndpoint` found [here](https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/llms/huggingface_endpoint.py) to power our chain - let's look at how we would implement it.

In [ ]:
from langchain.llms import HuggingFaceEndpoint

endpoint_url = (
    model_api_gateway
)

hf_llm = HuggingFaceEndpoint(
    endpoint_url=endpoint_url,
    huggingfacehub_api_token=os.environ["HF_TOKEN"],
    task="text-generation"
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Now we can use our endpoint like we would any other LLM!

In [ ]:
hf_llm.invoke("Hello, how are you?")

"\n\n- I'm good, thank you. How are you?\n\n- I'm good too, thank you. It's nice to meet you.\n\n- Nice to meet you too. Is there anything you would like to talk about or ask?\n\n- Actually, I was wondering if you could tell me more about your work. What do you do?\n\n- Of course! I work as a software engineer at a tech company. I'm responsible for developing and maintaining the software that our company uses.\n\n- That's really interesting. I've always been fascinated by technology and how it can be used to improve people's lives. What's your favorite part of your job?\n\n- I would say my favorite part is problem-solving. I enjoy working on complex issues and finding creative solutions. It's always challenging and rewarding to see a problem through to its conclusion.\n\n- I can imagine. It sounds like you have a really rewarding job. Do you have any advice for someone who wants to pursue a similar career path?\n\n- Yes, I do! I would say the most important thing is to have a strong fo

### HuggingFaceInferenceAPIEmbeddings

Now we can leverage the `HuggingFaceInferenceAPIEmbeddings` module in LangChain to connect to our Hugging Face Inference Endpoint hosted embedding model.

In [ ]:
embedding_api_gateway = "" # << Embedding Endpoint API URL

In [ ]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

embeddings_model = HuggingFaceInferenceAPIEmbeddings(api_key=os.environ["HF_TOKEN"], api_url=embedding_api_gateway)

In [ ]:
embeddings_model.embed_query("Hello, welcome to HF Endpoint Embeddings")[:10]

[0.02438463643193245,
 -0.03860687464475632,
 0.016107574105262756,
 -0.06647630035877228,
 0.06633372604846954,
 0.06500885635614395,
 -0.0022862316109240055,
 -0.003300266107544303,
 -0.014596718363463879,
 -0.04189097136259079]

#### ❓ Question #1

What is the embedding dimension of your selected embeddings model?

ANSWER: The dimensions of selected embeddings model (Snowflake-artic-embed-m) is 768.

## Task 6: Retrieving data from Arxiv

We'll leverage the `ArxivLoader` to load some papers about the "QLoRA" topic, and then split them into more manageable chunks!

In [ ]:
from langchain.document_loaders import ArxivLoader

docs = ArxivLoader(query="QLoRA", load_max_docs=5).load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 250,
    chunk_overlap = 0,
    length_function = len,
)

split_chunks = text_splitter.split_documents(docs)

In [ ]:
len(split_chunks)

1305

Just the same as we would with OpenAI's embeddings model - we can instantiate our `FAISS` vector store with our documents and our `HuggingFaceEmbeddings` model!

We'll need to take a few extra steps, though, due to a few limitations of the endpoint/FAISS.

We'll start by embeddings our documents in batches of `32`.

> NOTE: This process might take a while depending on the compute you assigned your embedding endpoint!

In [ ]:
embeddings = []
for i in range(0, len(split_chunks) - 1, 32):
  embeddings.append(embeddings_model.embed_documents([document.page_content for document in split_chunks[i:i+32]]))

In [ ]:
embeddings = [item for sub_list in embeddings for item in sub_list]

#### ❓ Question #2

Why do we have to limit our batches when sending to the Hugging Face endpoints

ANSWER: We limit our batches when sending to Hugging Face endpoints for efficiency, resource management, API limitation, data volume and cost.   

Now we can create text/embedding pairs which we want use to set-up our FAISS VectorStore!

In [ ]:
from langchain.vectorstores import FAISS

text_embedding_pairs = list(zip([document.page_content for document in split_chunks], embeddings))

faiss_vectorstore = FAISS.from_embeddings(text_embedding_pairs, embeddings_model)

Next, we set up FAISS as a retriever.

In [ ]:
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k" : 5})

Let's test it out!

In [ ]:
faiss_retriever.get_relevant_documents("What optimizer does QLoRA use?")

[Document(page_content='that are tuned by backpropagating gradients through\nthe quantized weights.\nQLORA reduces the average memory requirements\nof finetuning a 65B parameter model from >780GB\nof GPU memory to <48GB without degrading the'),
 Document(page_content='base model while decreasing their precision. This highlights the importance of efficiency benefits\nfrom QLORA. Since we did not observe performance degradation compared to full-finetuning in'),
 Document(page_content='nique. The results consistently show that QDy-\nLoRA achieves a superior performance by finding\nthe optimal rank.\nThe second experiment provides a more in-depth\ncomparison between QLoRA and QDyLoRA. In'),
 Document(page_content='of QDyLoRA through several instruct-fine-tuning\nTable 3: Comparing the performance of DyLoRA, QLoRA and QDyLoRA across different evaluation ranks. all'),
 Document(page_content='52.3\n72.6\n67.3\n63.0\nQLoRA w/ GPTQ\n4\n60.4\n52.5\n73.0\n67.2\n63.0\nQLoRA\n4\n59.8\n52.9\n75.0\n6

### Prompt Template

Now that we have our LLM and our Retiever set-up, let's connect them with our Prompt Template!

In [ ]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT_TEMPLATE = """\
Using the provided context, please answer the user's question. If you don't know, say you don't know.

Context:
{context}

Question:
{question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

#### ❓ Question #3

Does the ordering of the prompt matter?

ANSWER: It is possible that the order of the prompt may affect the operation.  For example, it may affect what the model's focus.  It may have impact from memory constaints.



## Task 7: Creating a simple RAG pipeline with LangChain v0.1.0

All that's left to do is set up a RAG chain - and away we go!

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

retrieval_augmented_qa_chain = (
    {
        "context": itemgetter("question") | faiss_retriever,
        "question": itemgetter("question"),
    }
    | rag_prompt
    | hf_llm
    | StrOutputParser()
)

Let's test it out!

In [ ]:
retrieval_augmented_qa_chain.invoke({"question" : "What is QLORA all about?"})

'\nAnswer:\nQLORA is a method that helps to close the resource gap between large corporations and small teams with consumer GPUs. It can be deployed to mobile phones and has the potential to significantly reduce the required memory for finetuning models. QLORA works by quantizing the weights of a model, which reduces the average memory requirements of finetuning a 65B parameter model from over 780GB of GPU memory to less than 48GB without degrading the performance of the model.'

# 🤝 Breakout Room #2

## Task 1: Set-up LangSmith

We'll be moving through this notebook to explain what visibility tools can do to help us!

Technically, all we need to do is set-up the next cell's environment variables!

In [ ]:
from uuid import uuid4

unique_id = uuid4().hex[0:8]

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE1 - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass('Enter your LangSmith API key: ')

Enter your LangSmith API key: ··········


Let's see what happens on the LangSmith project when we run this chain now!

In [ ]:
retrieval_augmented_qa_chain.invoke({"question" : "What is QLoRA all about?"})

'\nAnswer:\nQLoRA is a method that helps to close the resource gap between large corporations and small teams with consumer GPUs. It can be deployed to mobile phones and has the potential to significantly reduce the required memory for finetuning models. QLoRA works by quantizing the weights of a model, which reduces the average memory requirements of finetuning a 65B parameter model from >780GB of GPU memory to <48GB without degrading the performance of the model.'

We get *all of this information* for "free":

![image](https://i.imgur.com/8Wcpmcj.png)

> NOTE: We'll walk through this diagram in detail in class.

####🏗️ Activity #1:

Please describe the trace of the previous request and answer these questions:

The trace is awesome - it's like a protocol analyzer - It breaks it down into how the flow operates.  
High Level is a Runnable Sequence - Input/Output text stings.
Digging down into it - you see the Retriever with the chunks that was extracted that was used (passed to the LLM) to create the response.  Ah, so that's where it gets it from!

1. How many tokens did the request use?
311 prompt tokens.
75 completion tokens.

2. How long did the `HuggingFaceEndpoint` take to complete?
It took 6.42 seconds.

## Task 2: Creating a LangSmith dataset

Now that we've got LangSmith set-up - let's explore how we can create a dataset!

First, we'll create a list of questions!

In [ ]:
from langsmith import Client

questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
]

Now we can create our dataset through the LangSmith `Client()`.

In [ ]:
client = Client()
dataset_name = "QLoRA RAG Dataset v2"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about the QLoRA Paper to Evaluate RAG over the same paper."
)

client.create_examples(
    inputs=[{"question" : q} for q in questions],
    dataset_id=dataset.id
)

After this step you should be able to navigate to the following dataset in the LangSmith web UI.

![image](https://i.imgur.com/CdFYGTB.png)

## Task 3: Creating a custom evaluator

Now that we have a dataset - we can start thinking about evaluation.

We're going to make a `StringEvaluator` to measure "dopeness".

> NOTE: While this is a fun toy example - this can be extended to practically any use-case!

In [ ]:
import re
from typing import Any, Optional
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.evaluation import StringEvaluator

class DopenessEvaluator(StringEvaluator):
    """An LLM-based dopeness evaluator."""

    def __init__(self):
        llm = ChatOpenAI(model="gpt-4", temperature=0)

        template = """On a scale from 0 to 100, how dope (cool, awesome, lit) is the following response to the input:
        --------
        INPUT: {input}
        --------
        OUTPUT: {prediction}
        --------
        Reason step by step about why the score is appropriate, then print the score at the end. At the end, repeat that score alone on a new line."""

        self.eval_chain = PromptTemplate.from_template(template) | llm

    @property
    def requires_input(self) -> bool:
        return True

    @property
    def requires_reference(self) -> bool:
        return False

    @property
    def evaluation_name(self) -> str:
        return "scored_dopeness"

    def _evaluate_strings(
        self,
        prediction: str,
        input: Optional[str] = None,
        reference: Optional[str] = None,
        **kwargs: Any
    ) -> dict:
        evaluator_result = self.eval_chain.invoke(
            {"input": input, "prediction": prediction}, kwargs
        )
        reasoning, score = evaluator_result.content.split("\n", maxsplit=1)
        score = re.search(r"\d+", score).group(0)
        if score is not None:
            score = float(score.strip()) / 100.0
        return {"score": score, "reasoning": reasoning.strip()}

## Task 4: Initializing our evaluator config

Now we can initialize our `RunEvalConfig` which we can use to evaluate our chain against our dataset.

> NOTE: Check out the [documentation](https://docs.smith.langchain.com/evaluation/faq/custom-evaluators) for adding additional custom evaluators.

In [ ]:
from langchain.smith import RunEvalConfig, run_on_dataset

eval_config = RunEvalConfig(
    custom_evaluators=[DopenessEvaluator()],
    evaluators=[
        "criteria",
        RunEvalConfig.Criteria("harmfulness"),
        RunEvalConfig.Criteria(
            {
                "AI": "Does the response feel AI generated?"
                "Response Y if they do, and N if they don't."
            }
        ),
    ],
)

## Task 5: Evaluating our RAG pipeline

All that's left to do now is evaluate our pipeline!

In [ ]:
client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=retrieval_augmented_qa_chain,
    evaluation=eval_config,
    verbose=True,
    project_name="HF RAG Pipeline - Evaluation - v1",
    project_metadata={"version": "1.0.0"},
)

View the evaluation results for project 'HF RAG Pipeline - Evaluation - v1' at:
https://smith.langchain.com/o/4a61d98d-132a-5d07-84b0-e576bf509efa/datasets/cc559e46-2558-4cb7-a884-178a5a881334/compare?selectedSessions=8a829ef5-337a-40f3-a053-d65aa55d85d9

View all tests for Dataset QLoRA RAG Dataset v2 at:
https://smith.langchain.com/o/4a61d98d-132a-5d07-84b0-e576bf509efa/datasets/cc559e46-2558-4cb7-a884-178a5a881334
[------------------------------------------------->] 6/6
 Experiment Results:
        feedback.helpfulness  feedback.harmfulness  feedback.AI  feedback.scored_dopeness error  execution_time                                run_id
count                   6.00                  6.00         6.00                      6.00     0            6.00                                     6
unique                   NaN                   NaN          NaN                       NaN     0             NaN                                     6
top                      NaN                   NaN 

{'project_name': 'HF RAG Pipeline - Evaluation - v1',
 'results': {'12ab9fb0-3c3b-4ca1-8cf5-74ab7e314ef1': {'input': {'question': 'What optimizer is used in QLoRA?'},
   'feedback': [EvaluationResult(key='helpfulness', score=0, value='N', comment='The criterion for this task is "helpfulness". \n\nThe submission provides a detailed explanation of how the optimizer in QLoRA might work, mentioning backpropagating gradients through the quantized weights and fine-tuning the model. It also suggests possible optimizers that could be used, such as Adam, Adagrad, or Adadelta, based on their common use in deep learning and their ability to handle gradient updates in quantized weights. \n\nHowever, the submission does not definitively answer the question of what optimizer is used in QLoRA. The submitter admits that they do not know the specific optimizer used with certainty, as the context does not provide that information. \n\nWhile the submission is insightful and appropriate, it may not be con